In [177]:
import pandas as pd #pandas
import numpy as np #numpy
import math #for functions 
import os
import sys
from collections import defaultdict, deque
#from scipy.sparse import csr_matrix
#from sklearn.metrics.pairwise import cosine_similarity


print("Done")

Done


# FIRST BEGIN BY IMPORTING MOVIES AND CONVERTING TO DICT

In [178]:
movietitles = pd.read_csv('C:\CPS842 Project\Dataset\movie_titles.csv', header = None, names = ['ID', 'Year', 'Title'], encoding = 'ISO-8859-1').set_index('ID')

In [179]:
movietitles

,Year,Title
ID,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW
...,...,...
17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17767,2004.0,Fidel Castro: American Experience
17768,2000.0,Epoch


In [180]:
#put movie titles in dict
movieDict = movietitles.to_dict()


In [181]:
#get title of movie based on id
movieDict['Title'][46] 

'Rudolph the Red-Nosed Reindeer'

In [182]:
#get year of movie based on id
movieDict['Year'][46] #get year of release based on id

1964.0

In [183]:
#number of movies count
NUMBER_OF_MOVIES = len(movieDict["Title"])
NUMBER_OF_MOVIES

17770

# Now, we import the rating and user data set for each movie

In [184]:
%%time
# Load 
df = pd.read_csv('C:\CPS842 Project\Dataset\combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])

# Find empty rows to slice dataframe for each movie
tmp_movies = df[df['Rating'].isna()]['User'].reset_index()
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = df.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = df.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
rating = pd.concat(user_data)
del user_data, df, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(rating.shape))

rating["Rating"] = rating["Rating"].astype(np.int8)
rating["User"] = rating["User"].astype(np.int32)
rating["Movie"] = rating["Movie"].astype(np.int16)

del rating["Date"] #remove date as this is irrelavent
rating

Shape User-Ratings:	(24053764, 4)
Wall time: 33 s


,User,Rating,Movie
1,1488844,3,1
2,822109,5,1
3,885013,4,1
4,30878,4,1
5,823519,3,1
...,...,...,...
24058258,2591364,2,4499
24058259,1791000,2,4499
24058260,512536,5,4499
24058261,988963,3,4499


In [185]:
def changeName(x):
    return movieDict['Title'][x]

rating.Movie = rating.Movie.map(changeName)
rating

,User,Rating,Movie
1,1488844,3,Dinosaur Planet
2,822109,5,Dinosaur Planet
3,885013,4,Dinosaur Planet
4,30878,4,Dinosaur Planet
5,823519,3,Dinosaur Planet
...,...,...,...
24058258,2591364,2,In My Skin
24058259,1791000,2,In My Skin
24058260,512536,5,In My Skin
24058261,988963,3,In My Skin


In [186]:
l = rating["User"].value_counts()
l.head(100)


305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
           ... 
2602249     953
769702      945
402266      937
2297136     935
147386      932
Name: User, Length: 100, dtype: int64

In [187]:
rating['freq'] = rating['User'].map(rating['User'].value_counts())
rating['User'].value_counts()


305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
           ... 
791049        1
99015         1
47493         1
119728        1
2355355       1
Name: User, Length: 470758, dtype: int64

In [188]:
rating = rating.sort_values('freq', ascending=False)
rating

,User,Rating,Movie,freq
7094372,305344,2,Go Fish,4467
2051254,305344,2,Third World Cop,4467
23016000,305344,1,Manic,4467
2855641,305344,1,From Mao to Mozart: Isaac Stern in China,4467
14519183,305344,1,Man of La Mancha,4467
...,...,...,...,...
6399148,1580846,2,Better Off Dead,1
9178109,362123,4,U.S. Marshals,1
2143346,1039873,5,Hostage,1
21157462,1196769,5,Monk: Season 2,1


In [199]:
%%time
M = rating[:3000000].pivot_table(index=["User"], columns=["Movie"], values="Rating").fillna(0)
M.tail()

Wall time: 3.76 s


Movie,'N Sync: 'N the Mix,'N Sync: PopOdyssey Live,... And God Spoke,10,10 Minute Solution: Pilates,10 Things I Hate About You,100 Days Before the Command,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,...,Zatoichi the Outlaw,Zatoichi's Conspiracy,Zeus and Roxanne,Zhou Yu's Train,Ziegfeld Girl,Zig Zag: Special Edition,Zigzag,Zombie 3,Zorro,s-Cry-ed
User,,,,,,,,,,,,,,,,,,,,,
2647197,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2647888,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2648589,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2648869,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2648885,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [190]:
rating[:].groupby('Movie')['Rating'].mean().sort_values(ascending=False).head(20)

Movie
Lost: Season 1                                                                 4.670989
Ghost in the Shell: Stand Alone Complex: 2nd Gig                               4.586364
The Simpsons: Season 6                                                         4.581296
Inu-Yasha                                                                      4.554434
Lord of the Rings: The Return of the King: Extended Edition: Bonus Material    4.552000
Family Guy: Freakin' Sweet Collection                                          4.516007
The West Wing: Season 3                                                        4.473692
Six Feet Under: Season 4                                                       4.469693
Samurai Champloo                                                               4.467500
Inu-Yasha: The Movie 3: Swords of an Honorable Ruler                           4.464824
Stargate SG-1: Season 8                                                        4.461921
Gilmore Girls: Season 3   

In [191]:
def sim(m, n):
    ms = m - m.mean()
    ns = n - n.mean()
    return np.sum(ms * ns)/np.sqrt(np.sum(ms ** 2) * np.sum(ns ** 2))

In [192]:
%%time
sim(M['Gilmore Girls: Season 3'], M['Gilmore Girls: Season 3'])

Wall time: 4.02 ms


1.0

In [215]:
def top10(name):
    rec = []
    for title in M.columns:
        if title == name: #if title is same as arg, continue
            continue
            
        r = sim(M[name], M[title]) #calculate r value
        if np.isnan(r): #if r value doesnt make sense, move on
            continue
        else:
            rec.append((title, r)) #append to list
            
    rec.sort(key=lambda n: n[1], reverse=True)
    return rec[:10]

def top10user(name):
    rec = []
    movies = {}
    
    for user in M.index:
        if name == user: #if title is same as arg, continue
            continue
            
        r = sim(M.loc[name], M.loc[user]) #calculate r value
        if np.isnan(r): #if r value doesnt make sense, move on
            continue
        else:
            rec.append((user, r)) #append to lis
    
                   
    rec.sort(key=lambda n: n[1], reverse=True)
    
    for i in rec[:10]:
        count = 0
    
        for rating in M.loc[i[0]]:
            if rating != 0:
                
                #print(f"{i} rated {M.columns[count]} a {rating} ")
                
                if M.columns[count] in movies:
                    movies[M.columns[count]] += rating
                else:
                    movies[M.columns[count]] = rating
                
            count += 1
    
    sorted_movies = sorted(movies.items(), key=lambda x: x[1], reverse=True)
    
    count = 0
    for i in sorted_movies:
        if count == 15:
            break
        print(i[0], i[1]/10)
        count+=1
        
    
    return rec[:10]


In [216]:
%%time
top10user(2648885)
#similar = M.corrwith(M["Gilmore Girls: Season 3"]).head()
#similar

Lord of the Rings: The Fellowship of the Ring 4.9
Reservoir Dogs 4.8
The Silence of the Lambs 4.8
Braveheart 4.7
National Lampoon's Vacation 4.7
The Sixth Sense 4.7
American Beauty 4.6
Army of Darkness 4.6
Better Off Dead 4.6
Pirates of the Caribbean: The Curse of the Black Pearl 4.6
Alien: Collector's Edition 4.5
The Professional 4.5
Wayne's World 4.5
Aliens: Collector's Edition 4.4
Being John Malkovich 4.4
Wall time: 13.9 s


[(127903, 0.5487639462683419),
 (1066481, 0.5397442899387433),
 (1854193, 0.5363392550688387),
 (2438141, 0.5351515272477915),
 (2145441, 0.532871501311812),
 (990036, 0.5315892629560861),
 (508323, 0.5288205910412043),
 (631075, 0.5284877248874043),
 (682560, 0.5281833505714055),
 (827459, 0.5276261163951971)]

In [194]:
%%time
top10('Gilmore Girls: Season 3')

Wall time: 5.49 s


[('Gilmore Girls: Season 1', 0.7603227175671823),
 ('Roswell: Season 2', 0.2554333848452033),
 ('Buffy the Vampire Slayer: Season 6', 0.23480485278711904),
 ('Popular: Season 2', 0.22628995605192675),
 ('Angel: Season 4', 0.22238666323512146),
 ('Friends: Season 6', 0.21311932052367505),
 ('Dr. Quinn', 0.19621934701711663),
 ('Smallville: Season 1', 0.1957085203448753),
 ('The O.C.: Season 1', 0.19235711432426744),
 ('The Princess Diaries (Fullscreen)', 0.1892299094039427)]

In [195]:
M.loc[769].head

<bound method NDFrame.head of Movie
'N Sync: 'N the Mix            0.0
'N Sync: PopOdyssey Live       0.0
... And God Spoke              0.0
10                             0.0
10 Minute Solution: Pilates    0.0
                              ... 
Zig Zag: Special Edition       0.0
Zigzag                         0.0
Zombie 3                       0.0
Zorro                          0.0
s-Cry-ed                       0.0
Name: 769, Length: 4471, dtype: float64>

In [197]:
#print("test")
#rating.shape
#i = 0

#print(f"rowcount: {rating.shape}")

#def standardize(row): #this method standardizes the ratings
    
 #   new_row = (row - row.mean()) / (row.max() - row.min())
  #  return new_row

#ratings_std = rating.apply(standardize)
#movie_similarity = cosine_similarity(ratings_std)

M

Movie,'N Sync: 'N the Mix,'N Sync: PopOdyssey Live,... And God Spoke,10,10 Minute Solution: Pilates,10 Things I Hate About You,100 Days Before the Command,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,...,Zatoichi the Outlaw,Zatoichi's Conspiracy,Zeus and Roxanne,Zhou Yu's Train,Ziegfeld Girl,Zig Zag: Special Edition,Zigzag,Zombie 3,Zorro,s-Cry-ed
User,,,,,,,,,,,,,,,,,,,,,
769,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2213,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2976,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647197,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2647888,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2648589,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [204]:
count = 0
for rating in M.loc[769]:
    if rating != 0:
        print(f"rated {M.columns[count]} a {rating} ")
    count += 1

rated 10 Things I Hate About You a 2.0 
rated 28 Days Later a 4.0 
rated 301/302 a 3.0 
rated 50 First Dates a 3.0 
rated 9 1/2 Weeks a 3.0 
rated A Beautiful Mind a 4.0 
rated A Fistful of Dollars a 4.0 
rated A Night at the Opera a 3.0 
rated A Streetcar Named Desire a 3.0 
rated About Schmidt a 3.0 
rated About a Boy a 3.0 
rated Alien: Collector's Edition a 4.0 
rated Aliens: Collector's Edition a 4.0 
rated Along Came Polly a 2.0 
rated American Beauty a 3.0 
rated American Psycho a 4.0 
rated American Wedding a 3.0 
rated Amores Perros a 4.0 
rated Animal Crackers a 3.0 
rated Annie: Special Anniversary Edition a 3.0 
rated April Fool's Day a 3.0 
rated Arachnophobia a 2.0 
rated Army of Darkness a 3.0 
rated As Tears Go By a 4.0 
rated At Close Range a 3.0 
rated Bad Boys a 3.0 
rated Bananas a 4.0 
rated Barbarella a 2.0 
rated Batman Begins a 4.0 
rated Being John Malkovich a 4.0 
rated Bend It Like Beckham a 3.0 
rated Better Off Dead a 3.0 
rated Beverly Hills Cop a 4.0 
rat

In [ ]:
M.columns

In [ ]:
x = M.iloc[0] # get row of df





In [ ]:
x = M.loc[769]

for item,value in x.items():
 #   print(item, value)
    x[item] = 30
    
x
    

